In [ ]:
import pandas as pd
import transformers
import re
import torch
from datasets import Dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from huggingface_hub import interpreter_login
from sklearn.utils import resample
import os
from dotenv import load_dotenv

#### Acknowledgment
The code is adapted from unlslothai's work available at https://github.com/unslothai/unsloth

## Data sampling

### Generate confidence scores

In [ ]:
# load all remaining data (data not included in prompt selection or prompting experiment): text, label
data = pd.read_csv('')

In [ ]:
# load model
# Authenticate with Huggingface token
#!git config --global credential.helper store
interpreter_login()

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct" 

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto"
)

In [ ]:
# prompt for confidence scores
topic_v3_prompt_confidence = """
Du er en gennemsnitlig dansk nyhedsforbruger. Du får en artikel, og skal tildele den et passende antal kategorier svarende til de emner den omhandler. Artiklen omhandler oftest flere emner, men en kan også kun omhandle ét emne. 
Her er nogle generelle principper, du skal følge: Du skal anvende generel verdensviden og artiklens kontekst, hvor det er nødvendigt. Et emne skal bidrage med at fremføre artiklens væsentligste pointer for at kategorien tildeles. Det er muligt at samtagge flere kategorier, så to kategorier tilsammen skaber det emne, artiklen omhandler.
Kategorier: ”Begivenhed”: En hændelse, der finder sted - særligt for at markere noget vigtigt og evt. tilbagevendende. Herunder mærkedag, personlig begivenhed, sportsbegivenhed og underholdningsbegivenhed. ”Bolig”: Privatpersoners hjem. Herunder køb og salg, renovering/indretning og udlejning. ”Erhverv”: Emner relateret til beskæftigelsesmæssige forhold, inkl. arbejdspladser. Herunder ansættelsesforhold, offentlig instans og privat virksomhed. ”Dyr”: Levende flercellede organismer, ekskl. planter og mennesker. ”Katastrofe”: Begivenheder, som har negative konsekvenser for en større eller mindre gruppe (mennesker). Herunder mindre ulykke og større katastrofe. ”Kendt”: Personer, som ofte er alment kendt blandt den brede befolkning, fx deltagere i tv-programmer, politikere, sportspersonligheder, toperhvervsfolk og kongelige. ”Konflikt og krig”: Sammenstød, uoverensstemmelser eller stridigheder mellem to eller flere parter - evt. med voldelige konsekvenser. Herunder terror og væbnet konflit. ”Kriminalitet”: Ulovligheder, der omhandler fx dramatiske, dødelige eller sindsoprivende begivenheder, hvori politiet evt. har været involveret. Herunder bandekriminalitet, bedrageri og personfarlig kriminalitet. ”Kultur”: Den levevis, som er resultat af menneskelig aktivitet, og forestillingsverden, herunder skikke, holdninger og traditioner, ved en bestemt befolkningsgruppe i en bestemt periode. Herunder byliv, kunst, museum og seværdighed, or rejse. ”Livsstil”: Den bevidst valgte levemåde, som (en) privatperson(er) har - ofte for at signalere en bestemt identitet. Herunder erotik, familieliv, fritid, krop og velvære, mad og drikke og partnerskab. ”Politik”: Samfundsrelevante problemstillinger, værdier o.lign., som udføres på baggrund af en bestemt ideologi. Herunder international politik og national politik. ”Samfund”: Forhold for mennesker, der typisk deler samme geografiske område og/eller er knyttet sammen på anden vis, fx gennem indbyrdes afhængighed og tilknytning til nationalstat. Herunder bæredygtighed og klima, værdier, religion og tendens. ”Sport”: (Fysisk) aktivitet, som udøves individuelt eller på hold i professionelle sammenhænge, og hvor der ofte indgår specifikke regler og evt. udstyr. Herunder cykling, håndbold, fodbold, ketcher- og batsport og motorsport. ”Sundhed”: Helbredsmæssige forhold. Herunder kosmetisk behandling og sygdom og behandling. Teknologi”: Beskrivelser og anvendelser af tekniske hjælpemidler til udførelse af opgaver. Herunder forbrugerelektronik, kunstig intelligens og software. ”Transportmiddel”: Førbare anordninger, som kan fragte levende væsener og/eller artefakter. Herunder bil, mindre transportmiddel, offentlig transport og større transportmiddel. ”Uddannelse”: Strukturerede undervisningsforløb. Herunder grundskole, ungdomsuddannelse og videregående uddannelse. "Underholdning”: Forhold med evne til at more/frembringe glæde hos modtageren. Herunder litteratur, film og tv, musik og lyd og reality. ”Vejr”: Meterologiske forhold, fx forudsigelse eller afrapportering, ofte omhandlende et bestemt geografisk område på et bestemt tidspunkt. "Videnskab”: Metodologisk forskning og undersøgelse af bestemte dele af virkeligheden. Herunder naturvidenskab og samfundsvidenskab og humaniora. ”Økonomi”: Produktion, fordeling og forbrug (hos både privatpersoner og på statsligt niveau) af varer og tjenester. Herunder makroøkonomi og mikroøkonomi. 
Giv også en confidence score med to decimaler fra 0.00 til 1.00, der repræsenterer hvor sikker du er i din vurdering af sentiment, hvor 0 er meget usikker og 1 er meget sikker.
Giv et præcist svar i json: {{topics: [”kategori”, ”kategori”, ”kategori”, ... ], "confidence": "score"}}. Det er meget vigtigt, at du kun returnerer dette format.
"""

In [ ]:
# Function to run Llama with few-shot prompts
def fewshot_topic_annotation(text, prompt, fewshot_dataset):
    # Sample few-shot examples from dataset without current article
    fewshot_dataset = fewshot_dataset[fewshot_dataset['text'] != text]
    fewshot_examples = fewshot_dataset.sample(1).to_dict(orient='records')
    # Create few-shot part of prompt
    fewshot_examples = "\n".join([f"Artikel: {example['text']}. Artiklen omhandler dette/disse emne(r): {example['label']}." for example in fewshot_examples])
    
    messages = [
    {"role": "system", "content": f"{prompt}. Her er et eksempel på en artikel og dens emner: {fewshot_examples}"},
    {"role": "user", "content": f"Artikel: {text}. Artiklen omhandler dette/disse emne(r): "},
    ]

    outputs = pipeline(
        messages,
        max_new_tokens=256,
    )
    
    # Return the generated content
    return outputs[0]["generated_text"][-1]["content"]

In [ ]:
# Apply few-shot
data["llm_annotation"] = data["text"].apply(lambda text: fewshot_topic_annotation(text, prompt=topic_v3_prompt_confidence, fewshot_dataset=data))

In [ ]:
# extract confidence scores
def extract_confidence(value):
    match = re.search(r'"confidence":\s*"?(\d+\.\d+)"?', str(value))
    if match:
        return int(float(match.group(1)))
    return None  # Return None if no match is found

data["confidence_scores"] = data["llm_annotation"].apply(extract_confidence)

In [ ]:
# Format topics from strings to lists
def annotation_to_list(annotation):
    annotation_list = []
    
    topics = ["Begivenhed", "Bolig", "Erhverv", "Dyr", "Katastrofe", "Kendt", "Konflikt og krig", "Kriminalitet", "Kultur", "Livsstil",
          "Politik", "Samfund", "Sport", "Sundhed", "Teknologi", "Transportmiddel", "Uddannelse", "Underholdning", "Vejr", "Videnskab", "Økonomi"]

    # Regular expression to capture words inside single/double quotes or just words
    annotation_topics = [topic.lower() for topic in re.findall(r'["\']?([a-zA-ZæøåÆØÅ\s]+)["\']?', annotation)]
    
    # Iterate over the topics and check for case-insensitive matches in the extracted topics
    for topic in topics:
        if topic.lower() in annotation_topics:
            annotation_list.append(topic)
    
    return annotation_list

data["label"] = data["label"].apply(annotation_to_list)
data["llm_annotation"] = data["llm_annotation"].apply(annotation_to_list)

### Random sampling

In [ ]:
data = data.sample(n=300)

### Selective sampling

In [ ]:
# Define bins and labels for confidence intervals
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ["0-0.1", "0.1-0.2", "0.2-0.3", "0.3-0.4", "0.4-0.5", "0.5-0.6", "0.6-0.7", "0.7-0.8", "0.8-0.9", "0.9-1.0"]
data["confidence_interval"] = pd.cut(data["confidence_scores"], bins=bins, labels=labels, include_lowest=True)

# Determine correctness of LLM annotations
def classify_correctness(row):
    human_set = set(row["label"])
    llm_set = set(row["llm_annotation"])
    if human_set == llm_set:
        return "correct"
    elif human_set & llm_set:
        return "partially_correct"
    else:
        return "incorrect"

data["is_correct"] = data.apply(classify_correctness, axis=1)

# Calculate proportions for each group based on available data
group_counts = data.groupby(["confidence_interval", "is_correct"]).size()
total_available = group_counts.sum()
group_proportions = group_counts / total_available

# Calculate target sample sizes based on the proportions
total_samples = 300
sample_sizes = (group_proportions * total_samples).round().astype(int)

# Initialize an empty DataFrame for the sampled data
sampled_df = pd.DataFrame()

# Sample examples from each group without replacement
for (interval, correct), size in sample_sizes.items():
    if size > 0:
        # Subset data for this group
        group_data = data[(data["confidence_interval"] == interval) & (data["is_correct"] == correct)]
        
        # Sample without replacement, up to the size of the group
        if len(group_data) >= size:
            sampled_group = group_data.sample(n=size, random_state=42, replace=False)
        else:
            # If not enough examples, sample all available examples
            sampled_group = group_data
            print(f"Warning: Not enough examples in group ({interval}, {correct}). Adding all {len(group_data)} examples.")
        
        sampled_df = pd.concat([sampled_df, sampled_group])

# Reset index for the sampled DataFrame
sampled_df.reset_index(drop=True, inplace=True)

data = sampled_df

## Fine-tune Llama

In [ ]:
max_seq_length = 5000 
dtype = None 
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = os.getenv("HUGGINGFACE_TOKEN"),
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = True, 
    loftq_config = None,
)

In [ ]:
# format fine-tuning data to chat template
data["text"] = data["messages"].apply(lambda chat: tokenizer.apply_chat_template(chat, tokenize=False))

# convert to Huggingface dataset
data_dict = {"text": data["text"].tolist()}
dataset = Dataset.from_dict(data_dict)

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        warmup_steps = 10,
        max_steps = 100,
        learning_rate = 1e-5, 
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

## Inference with fine-tuned model

In [ ]:
# Load evaluation data
evaluation_data = pd.read_csv("")

In [ ]:
# Load model
model = FastLanguageModel.for_inference(model)

In [ ]:
# V0 zeroshot 
def run_v0_topic_inference(text, model):

    # Define the chat input, inserting the text. v3 prompt    
    chat_input = [
        f'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n'
        f'Cutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n'
        f'Du er en gennemsnitlig dansk nyhedsforbruger. Du får en artikel, og skal tildele den et passende antal kategorier svarende til de emner den omhandler.'
        f'Artiklen omhandler oftest flere emner, men den kan også omhandle kun et emne. '
        f'Kategorier: ”Begivenhed”, ”Bolig”, ”Erhverv”, ”Dyr”, ”Katastrofe”, ”Kendt”, ”Konflikt og krig”, ”Kriminalitet”, ”Kultur”, ”Livsstil”, ”Politik”, ”Samfund”, ”Sport”, ”Sundhed”, ”Teknologi”, ”Transportmiddel”, ”Uddannelse”, ”Underholdning”, ”Vejr”, ”Videnskab”, ”Økonomi”. '
        f'Giv et præcist svar i json: {{emner: [”kategori”, ”kategori”, ”kategori”, ... ]}}.<|eot_id|><|start_header_id|>user<|end_header_id>\n\n'
        f'Artikel: {text} \nArtiklen omhandler dette/disse emne(r):<|eot_id|><|start_header_id|>assistant<|end_header_id>\n\n'
    ]

    # Tokenize the chat input for the model
    inputs = tokenizer(chat_input, return_tensors="pt").to("cuda")

    # Generate the output
    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

    # Decode the generated output
    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return decoded_outputs

evaluation_data["v0-zeroshot-annotation"] = evaluation_data["text"].apply(lambda text: run_v0_topic_inference(text, model=model))

In [ ]:
# V3 few-shot
def run_v3_fewshot_topic_inference(text, model, fewshot_dataset):
    # Sample few-shot examples from dataset without current article
    fewshot_dataset = fewshot_dataset[fewshot_dataset['text'] != text]
    fewshot_examples = fewshot_dataset.sample(1).to_dict(orient='records')
    # Create few-shot part of prompt
    fewshot_example = "\n".join([f"Artikel: {example['text']}. Artiklen omhandler dette/disse emne(r): {example['label']}." for example in fewshot_examples])
    
    # Define the chat input, inserting the text. v3 prompt    
    chat_input = [
        f'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n'
        f'Cutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n'
        f'Du er en gennemsnitlig dansk nyhedsforbruger. Du får en artikel, og skal tildele den et passende antal kategorier svarende til de emner den omhandler. Artiklen omhandler oftest flere emner, men en kan også kun omhandle ét emne.'
        f'Her er nogle generelle principper, du skal følge: Du skal anvende generel verdensviden og artiklens kontekst, hvor det er nødvendigt. Et emne skal bidrage med at fremføre artiklens væsentligste pointer for at kategorien tildeles. Det er muligt at samtagge flere kategorier, så to kategorier tilsammen skaber det emne, artiklen omhandler.'
        f'Kategorier: ”Begivenhed”: En hændelse, der finder sted - særligt for at markere noget vigtigt og evt. tilbagevendende. Herunder mærkedag, personlig begivenhed, sportsbegivenhed og underholdningsbegivenhed. ”Bolig”: Privatpersoners hjem. Herunder køb og salg, renovering/indretning og udlejning. ”Erhverv”: Emner relateret til beskæftigelsesmæssige forhold, inkl. arbejdspladser. Herunder ansættelsesforhold, offentlig instans og privat virksomhed. ”Dyr”: Levende flercellede organismer, ekskl. planter og mennesker. ”Katastrofe”: Begivenheder, som har negative konsekvenser for en større eller mindre gruppe (mennesker). Herunder mindre ulykke og større katastrofe. ”Kendt”: Personer, som ofte er alment kendt blandt den brede befolkning, fx deltagere i tv-programmer, politikere, sportspersonligheder, toperhvervsfolk og kongelige. ”Konflikt og krig”: Sammenstød, uoverensstemmelser eller stridigheder mellem to eller flere parter - evt. med voldelige konsekvenser. Herunder terror og væbnet konflit. ”Kriminalitet”: Ulovligheder, der omhandler fx dramatiske, dødelige eller sindsoprivende begivenheder, hvori politiet evt. har været involveret. Herunder bandekriminalitet, bedrageri og personfarlig kriminalitet. ”Kultur”: Den levevis, som er resultat af menneskelig aktivitet, og forestillingsverden, herunder skikke, holdninger og traditioner, ved en bestemt befolkningsgruppe i en bestemt periode. Herunder byliv, kunst, museum og seværdighed, or rejse. ”Livsstil”: Den bevidst valgte levemåde, som (en) privatperson(er) har - ofte for at signalere en bestemt identitet. Herunder erotik, familieliv, fritid, krop og velvære, mad og drikke og partnerskab. ”Politik”: Samfundsrelevante problemstillinger, værdier o.lign., som udføres på baggrund af en bestemt ideologi. Herunder international politik og national politik. ”Samfund”: Forhold for mennesker, der typisk deler samme geografiske område og/eller er knyttet sammen på anden vis, fx gennem indbyrdes afhængighed og tilknytning til nationalstat. Herunder bæredygtighed og klima, værdier, religion og tendens. ”Sport”: (Fysisk) aktivitet, som udøves individuelt eller på hold i professionelle sammenhænge, og hvor der ofte indgår specifikke regler og evt. udstyr. Herunder cykling, håndbold, fodbold, ketcher- og batsport og motorsport. ”Sundhed”: Helbredsmæssige forhold. Herunder kosmetisk behandling og sygdom og behandling. Teknologi”: Beskrivelser og anvendelser af tekniske hjælpemidler til udførelse af opgaver. Herunder forbrugerelektronik, kunstig intelligens og software. ”Transportmiddel”: Førbare anordninger, som kan fragte levende væsener og/eller artefakter. Herunder bil, mindre transportmiddel, offentlig transport og større transportmiddel. ”Uddannelse”: Strukturerede undervisningsforløb. Herunder grundskole, ungdomsuddannelse og videregående uddannelse. "Underholdning”: Forhold med evne til at more/frembringe glæde hos modtageren. Herunder litteratur, film og tv, musik og lyd og reality. ”Vejr”: Meterologiske forhold, fx forudsigelse eller afrapportering, ofte omhandlende et bestemt geografisk område på et bestemt tidspunkt. "Videnskab”: Metodologisk forskning og undersøgelse af bestemte dele af virkeligheden. Herunder naturvidenskab og samfundsvidenskab og humaniora. ”Økonomi”: Produktion, fordeling og forbrug (hos både privatpersoner og på statsligt niveau) af varer og tjenester. Herunder makroøkonomi og mikroøkonomi. '
        f'Her er et eksempel på en artikel og emner: {fewshot_example}'
        f'Giv et præcist svar i json: {{topics: [”kategori”, ”kategori”, ”kategori”, ... ]}}.<|eot_id|><|start_header_id|>user<|end_header_id>\n\n'
        f'Artikel: {text} \nArtiklen omhandler dette/disse emne(r):<|eot_id|><|start_header_id|>assistant<|end_header_id>\n\n'
    ]

    # Tokenize the chat input for the model
    inputs = tokenizer(chat_input, return_tensors="pt").to("cuda")

    # Generate the output
    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

    # Decode the generated output
    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return decoded_outputs

evaluation_data["v3-fewshot-annotation"] = evaluation_data["text"].apply(lambda text: run_v3_fewshot_topic_inference(text, model=model, fewshot_dataset=evaluation_data))

In [ ]:
# Extract topics from LLM output
def extract_topics(target, reference):
    index = reference.find(target)
    if index == -1:
        # Target string not found
        return ""
    # Return everything after the target string
    return reference[index + len(target):]

# Apply the function to the first element ([0]) of each list in the Series
evaluation_data["llm_annotation_v3-fewshot-annotation"] = evaluation_data["v3-fewshot-annotation"].apply(
    lambda x: extract_topics("assistant<|end_header_id>", x[0]) if isinstance(x, list) and len(x) > 0 else ""
)

evaluation_data["llm_annotation_v0-zeroshot-annotation"] = evaluation_data["v0-zeroshot-annotation"].apply(
    lambda x: extract_topics("assistant<|end_header_id>", x[0]) if isinstance(x, list) and len(x) > 0 else ""
)

In [ ]:
# save evaluation results in csv
evaluation_data.to_csv("")